In [ ]:
import os
import sys  
import json
import torch 
import argparse
import numpy as np
from PIL import Image 
from tqdm import tqdm
from datasets import load_dataset
from utils import model_gen, extract_answer
from transformers import AutoModelForCausalLM, AutoTokenizer  

import openai
openai.api_key=''

dataset = load_dataset("AI4Math/MathVista")

ckpt_path = 'internlm/internlm-xcomposer2-vl-7b'
tokenizer = AutoTokenizer.from_pretrained(ckpt_path, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(ckpt_path, device_map="cuda", trust_remote_code=True).eval().cuda().to(torch.bfloat16)
model.tokenizer = tokenizer

In [ ]:
final = {}
for d in tqdm(dataset["testmini"]):
    q = d['query'] 
    text = f"[UNUSED_TOKEN_146]user\n{q}[UNUSED_TOKEN_145]\n[UNUSED_TOKEN_146]assistant\n" 
    image = d['decoded_image']
    with torch.cuda.amp.autocast():
        with torch.no_grad():
            response = model_gen(model, text, image) 
            final[d['pid']] = response 

In [ ]:
full_out = {}
for d in tqdm(dataset["testmini"]): 
    res = extract_answer(final[d['pid']], d) 
    d['extraction'] = res
    d['decoded_image'] = ''
    full_out[d['pid']] = d 

In [10]:
json.dump(full_out, open( 'MathVista_Testmini_InternLM_XComposer_VL.json', 'w'))